## Importation des données

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv("final_train.csv", index_col=[0])
test = pd.read_csv("final_test.csv", index_col=[0])

In [ ]:
train.head()

## Séparation et Encodage

In [ ]:
# Séparation des données en train (80%) et test (20%)
X = train[["PassengerId", "Solo", "FamilySize", "HomePlanet", "CryoSleep", 
           "CabinDeck", "CabinNum", "CabinSide", "Destination", "Age", 
           "AgeRange", "VIP", "RoomService", "FoodCourt", "ShoppingMall", 
           "Spa", "VRDeck", "Expenses", "Spending", "Name"]]
y = train["Transported"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [ ]:
# Création une correspondance entre les noms uniques et des entiers uniques
names = pd.concat([X, test], axis=0)['Name'].unique()
name_to_int = {name: i for i, name in enumerate(names)}

# Mise en correspondance la colonne "Name" 
X_train['Name'] = X_train['Name'].map(name_to_int)
X_test['Name'] = X_test['Name'].map(name_to_int)
test['Name'] = test['Name'].map(name_to_int)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Instanciation des objets LabelEncoder
le_homeplanet = LabelEncoder()
le_cabindeck = LabelEncoder()
le_cabinside = LabelEncoder()
le_destination = LabelEncoder()

# Transformation des variables qualitatives en numériques à l'aide de label encoding
X_train['HomePlanet'] = le_homeplanet.fit_transform(X_train['HomePlanet'].astype(str))
X_train['CabinDeck'] = le_cabindeck.fit_transform(X_train['CabinDeck'].astype(str))
X_train['CabinSide'] = le_cabinside.fit_transform(X_train['CabinSide'].astype(str))
X_train['Destination'] = le_destination.fit_transform(X_train['Destination'].astype(str))

# Transformation des données de test
X_test['HomePlanet'] = le_homeplanet.transform(X_test['HomePlanet'].astype(str))
X_test['CabinDeck'] = le_cabindeck.transform(X_test['CabinDeck'].astype(str))
X_test['CabinSide'] = le_cabinside.transform(X_test['CabinSide'].astype(str))
X_test['Destination'] = le_destination.transform(X_test['Destination'].astype(str))

# Transformation des données de test pour submissions
test['HomePlanet'] = le_homeplanet.transform(test['HomePlanet'].astype(str))
test['CabinDeck'] = le_cabindeck.transform(test['CabinDeck'].astype(str))
test['CabinSide'] = le_cabinside.transform(test['CabinSide'].astype(str))
test['Destination'] = le_destination.transform(test['Destination'].astype(str))

In [ ]:
X_train.head()

## Recherche des hyperparamètres

### Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Paramètres à tester pour le réglage des hyperparamètres
param_grid = 
{
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto']
}

# Initialisation du modèle SVM
svm = SVC(random_state=42)

# Initialisation GridSearchCV pour ajuster les hyperparamètres du modèle
grid_search = GridSearchCV(svm, param_grid, cv=5)

# Fit du modèle en utilisant les données d'entraînement
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres trouvés
print("Best hyperparameters: ", grid_search.best_params_)

# Sauvegarde des meilleurs hyperparamètres trouvés
svm_best_param = grid_search.best_params_

# Instanciation du modèle avec les meilleurs hyperparamètres
svm_optimized = SVC(**grid_search.best_params_)

# Entraînement du modèle sur les données d'entraînement
svm_optimized.fit(X_train, y_train)

# Utilisation des meilleurs hyperparamètres pour faire des prédictions sur les données de test
y_pred = svm_optimized.predict(X_test)

# Calcul de l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy SVM:", accuracy)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Paramètres à tester pour le réglage des hyperparamètres
param_grid = 
{
    'n_estimators': [100, 200, 300],
    'max_depth': [30, 50, None],
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

# Initialisation du modèle Random Forest
random_forest = RandomForestClassifier(random_state=42)

# Initialisation GridSearchCV pour ajuster les hyperparamètres du modèle
grid_search = GridSearchCV(random_forest, param_grid, cv=5)

# Fit du modèle en utilisant les données d'entraînement
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres trouvés
print("Best parameters : ", grid_search.best_params_)

# Sauvegarde des meilleurs hyperparamètres trouvés
rf_best_param = grid_search.best_params_

# Instanciation du modèle avec les meilleurs hyperparamètres
random_forest_optimized = RandomForestClassifier(**grid_search.best_params_)

# Entraînement du modèle sur les données d'entraînement
random_forest_optimized.fit(X_train, y_train)

# Utilisation des meilleurs hyperparamètres pour faire des prédictions sur les données de test
y_pred = random_forest_optimized.predict(X_test)

# Calcul de l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Random Forest :", accuracy)

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Paramètres à tester pour le réglage des hyperparamètres
param_grid = 
{
    'learning_rate': [1, 0.1, 0.01],
    'max_depth': [5, 6, 7],
    'max_leaf_nodes': [8, 9, 10],
    'min_impurity_decrease': [0.004, 0.005, 0.006],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [100, 200, 300],
    'n_iter_no_change': [5, 6, 7]
}

# Initialisation du modèle Gradient Boosting
gbc = GradientBoostingClassifier(random_state=42)

# Initialiser GridSearchCV pour ajuster les hyperparamètres du modèle
grid_search = GridSearchCV(gbc, param_grid, cv=5)

# Fit du modèle en utilisant les données d'entraînement
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres : ", grid_search.best_params_)

# Sauvegarde des meilleurs hyperparamètres trouvés
gbc_best_param = grid_search.best_params_

# Instanciation du modèle avec les meilleurs hyperparamètres
gbc_optimized = GradientBoostingClassifier(**grid_search.best_params_)

# Entraînement du modèle sur les données d'entraînement
gbc_optimized.fit(X_train, y_train)

# Utilisation des meilleurs hyperparamètres pour faire des prédictions sur les données de test
y_pred = gbc_optimized.predict(X_test)

# Calcul de l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Gradient Boosting :", accuracy)

### XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Paramètres à tester pour le réglage des hyperparamètres
param_grid = 
{
    'learning_rate': [0.01, 0.02, 0.03],
    'max_depth': [305, 307, 310],
    'min_child_weight': [4, 5, 6],
    'gamma': [4, 5, 6],
    'n_estimators': [780, 790, 800],
    'alpha': [0.011, 0.012, 0.013]
}

# Initialisation du modèle XGBoost
xgb = XGBClassifier(random_state=42)

# Initialiser GridSearchCV pour ajuster les hyperparamètres du modèle
grid_search = GridSearchCV(xgb, param_grid, cv=5)

# Fit du modèle en utilisant les données d'entraînement
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres : ", grid_search.best_params_)

# Sauvegarde des meilleurs hyperparamètres trouvés
xgb_best_param = grid_search.best_params_

# Instanciation du modèle avec les meilleurs hyperparamètres
xgb_optimized = xgb.GradientBoostingClassifier(**grid_search.best_params_)

# Entraînement du modèle sur les données d'entraînement
xgb_optimized.fit(X_train, y_train)

# Utilisation des meilleurs hyperparamètres pour faire des prédictions sur les données de test
y_pred = xgb_optimized.predict(X_test)

# Calcul de l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy XGBoost :", accuracy)

### Ada Boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

# Paramètres à tester pour le réglage des hyperparamètres
param_grid = 
{
    'n_estimators': [340, 350, 360],
    'learning_rate': [0.1, 0.05, 0.15]
}

# Initialisation du modèle Ada Boost
ada_boost = AdaBoostClassifier(random_state=42)

# Initialisation GridSearchCV pour ajuster les hyperparamètres du modèle
grid_search = GridSearchCV(ada_boost, param_grid, cv=5)

# Fit du modèle en utilisant les données d'entraînement
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres trouvés
print("Best parameters : ", grid_search.best_params_)

# Sauvegarde des meilleurs hyperparamètres trouvés
ada_best_param = grid_search.best_params_

# Instanciation du modèle avec les meilleurs hyperparamètres
ada_optimized = AdaBoostClassifier(**grid_search.best_params_)

# Entraînement du modèle sur les données d'entraînement
ada_optimized.fit(X_train, y_train)

# Utilisation des meilleurs hyperparamètres pour faire des prédictions sur les données de test
y_pred = ada_optimized.predict(X_test)

# Calcul de l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Ada Boost :", accuracy)

## Sélection des variables

In [ ]:
# Concaténation des données d'entraînement et de test
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

# Sélection des caractéristiques en utilisant la régression logistique avec une régularisation L1
lr = LogisticRegression(penalty='l1', solver='liblinear', C=0.1, random_state=0)
selector = SelectFromModel(lr)
selector.fit(X, y)
selected_features = X.columns[selector.get_support()]

In [ ]:
selected_features

## Prédiction à l'aide des modèles

### Support Vector Machine

In [ ]:
# Fit du modèle sur l'ensemble des données
svm_optimized.fit(X[selected_features], y)
# Prédiction avec le modèle
svm_pred = svm_optimized.predict(test[selected_features])
# Création d'un dataFrame à partir des identifiants des passagers et des prédictions
result_svm = pd.DataFrame({'PassengerId': test['PassengerId'], 'Transported': svm_pred})
# Transformation des valeurs booléennes en chaînes de caractères "True" ou "False"
result_svm['Transported'] = result_svm['Transported'].astype(str)
result_svm['Transported'] = result_svm['Transported'].replace({'0': 'False', '1': 'True'})
# Sauvegarde des résultats dans un fichier CSV
result_svm.to_csv('submissions_svm.csv', index=False, columns=['PassengerId', 'Transported'])

### Random Forest

In [ ]:
# Fit du modèle sur l'ensemble des données
random_forest_optimized.fit(X[selected_features], y)
# Prédiction avec le modèle
random_forest_pred = random_forest_optimized.predict(test[selected_features])
# Création d'un dataFrame à partir des identifiants des passagers et des prédictions
result_random_forest = pd.DataFrame({'PassengerId': test['PassengerId'], 'Transported': random_forest_pred})
# Transformation des valeurs booléennes en chaînes de caractères "True" ou "False"
result_random_forest['Transported'] = result_random_forest['Transported'].astype(str)
result_random_forest['Transported'] = result_random_forest['Transported'].replace({'0': 'False', '1': 'True'})
# Sauvegarde des résultats dans un fichier CSV
result_random_forest.to_csv('submissions_random_forest.csv', index=False, columns=['PassengerId', 'Transported'])

### Gradient Boosting Classifier

In [ ]:
# Fit du modèle sur l'ensemble des données
gbc_optimized.fit(X[selected_features], y)
# Prédiction avec le modèle
gbc_pred = gbc_optimized.predict(test[selected_features])
# Création d'un dataFrame à partir des identifiants des passagers et des prédictions
result_gbc = pd.DataFrame({'PassengerId': test['PassengerId'], 'Transported': gbc_pred})
# Transformation des valeurs booléennes en chaînes de caractères "True" ou "False"
result_gbc['Transported'] = result_gbc['Transported'].astype(str)
result_gbc['Transported'] = result_gbc['Transported'].replace({'0': 'False', '1': 'True'})
# Sauvegarde des résultats dans un fichier CSV
result_gbc.to_csv('submissions_gbc.csv', index=False, columns=['PassengerId', 'Transported'])

### XGBoost

In [ ]:
# Fit du modèle sur l'ensemble des données
xgb_optimized.fit(X[selected_features], y)
# Prédiction avec le modèle
xgb_pred = xgb_optimized.predict(test[selected_features])
# Création d'un dataFrame à partir des identifiants des passagers et des prédictions
result_xgb = pd.DataFrame({'PassengerId': test['PassengerId'], 'Transported': xgb_pred})
# Transformation des valeurs booléennes en chaînes de caractères "True" ou "False"
result_xgb['Transported'] = result_xgb['Transported'].astype(str)
result_xgb['Transported'] = result_xgb['Transported'].replace({'0': 'False', '1': 'True'})
# Sauvegarde des résultats dans un fichier CSV
result_xgb.to_csv('submissions_xgb.csv', index=False, columns=['PassengerId', 'Transported'])

### Ada Boost

In [ ]:
# Fit du modèle sur l'ensemble des données
ada_optimized.fit(X[selected_features], y)
# Prédiction avec le modèle
ada_pred = ada_optimized.predict(test[selected_features])
# Création d'un dataFrame à partir des identifiants des passagers et des prédictions
result_ada = pd.DataFrame({'PassengerId': test['PassengerId'], 'Transported': ada_pred})
# Transformation des valeurs booléennes en chaînes de caractères "True" ou "False"
result_ada['Transported'] = result_ada['Transported'].astype(str)
result_ada['Transported'] = result_ada['Transported'].replace({'0': 'False', '1': 'True'})
# Sauvegarde des résultats dans un fichier CSV
result_ada.to_csv('submissions_ada.csv', index=False, columns=['PassengerId', 'Transported'])